In [32]:
import unicodedata
import re
import pandas as pd

In [33]:
# https://cs.wikipedia.org/wiki/České_předložky

TXT = """ 
<dl><dd>s – s tebou</dd>
<dd>za – stát za rohem, kdo za tím je?</dd>
<dd>před – přímo před tebou, před mnoha lety, uniknout před pronásledovateli, varovat před ním</dd>
<dd>pod – ležet pod postelí</dd>
<dd>nad – viset nad pohovkou</dd>
<dd>mezi – nacházet se mezi póly</dd></dl>
<div class="mw-heading mw-heading3"><h3 id="Genitiv_(2._pád)"><span id="Genitiv_.282._p.C3.A1d.29"></span><a href="/wiki/Genitiv" title="Genitiv">Genitiv</a> (2. pád)</h3><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/w/index.php?title=%C4%8Cesk%C3%A9_p%C5%99edlo%C5%BEky&amp;veaction=edit&amp;section=5" title="Editace sekce: Genitiv (2. pád)" class="mw-editsection-visualeditor"><span>editovat</span></a><span class="mw-editsection-divider"> | </span><a href="/w/index.php?title=%C4%8Cesk%C3%A9_p%C5%99edlo%C5%BEky&amp;action=edit&amp;section=5" title="Editovat zdrojový kód sekce Genitiv (2. pád)"><span>editovat zdroj</span></a><span class="mw-editsection-bracket">]</span></span></div>
<dl><dd>od – odstup od něj</dd>
<dd>z – kouř z komínu</dd>
<dd>z – z police<sup id="cite_ref-1" class="reference"><a href="#cite_note-1"><span class="cite-bracket">[</span>1<span class="cite-bracket">]</span></a></sup><sup id="cite_ref-2" class="reference"><a href="#cite_note-2"><span class="cite-bracket">[</span>2<span class="cite-bracket">]</span></a></sup></dd>
<dd>do – do bytu, čekat do tří</dd>
<dd>bez – bez dcerky neodejdu</dd>
<dd>krom(ě) – kromě něj tam byla i ona</dd>
<dd>místo – místo tebe hrál náhradník</dd>
<dd>podle – podle normy</dd>
<dd>podél/kolem – šel kolem stolu</dd>
<dd>okolo – tráva okolo studny</dd>
<dd>u – u stolu</dd>
<dd>vedle – na té fotce stojí Lucie vedle Moniky</dd>
<dd>během – během prázdnin</dd>
<dd>pomocí – pomocí klacku ho dostali z bažiny</dd>
<dd>stran – domluvit se stran ceny</dd>
<dd>prostřednictvím – informovat prostřednictvím televizního zpravodajství</dd>
<dd>za – bylo to za války</dd>
<dd>vinou – vinou požáru jsme přišli o všechno</dd></dl>
<div class="mw-heading mw-heading3"><h3 id="Dativ_(3._pád)"><span id="Dativ_.283._p.C3.A1d.29"></span><a href="/wiki/Dativ" title="Dativ">Dativ</a> (3. pád)</h3><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/w/index.php?title=%C4%8Cesk%C3%A9_p%C5%99edlo%C5%BEky&amp;veaction=edit&amp;section=6" title="Editace sekce: Dativ (3. pád)" class="mw-editsection-visualeditor"><span>editovat</span></a><span class="mw-editsection-divider"> | </span><a href="/w/index.php?title=%C4%8Cesk%C3%A9_p%C5%99edlo%C5%BEky&amp;action=edit&amp;section=6" title="Editovat zdrojový kód sekce Dativ (3. pád)"><span>editovat zdroj</span></a><span class="mw-editsection-bracket">]</span></span></div>
<dl><dd>k – jdu k tobě</dd>
<dd>(na)proti – být proti návrhu, je to naproti lékárně</dd>
<dd>oproti – oproti teoriím věřím faktům</dd>
<dd>kvůli – udělej to kvůli mně</dd>
<dd>díky – díky němu máme své peníze zpět</dd>
<dd>vůči – vůči němu to není zdvořilé</dd></dl>
<div class="mw-heading mw-heading3"><h3 id="Akuzativ_(4._pád)"><span id="Akuzativ_.284._p.C3.A1d.29"></span><a href="/wiki/Akuzativ" title="Akuzativ">Akuzativ</a> (4. pád)</h3><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/w/index.php?title=%C4%8Cesk%C3%A9_p%C5%99edlo%C5%BEky&amp;veaction=edit&amp;section=7" title="Editace sekce: Akuzativ (4. pád)" class="mw-editsection-visualeditor"><span>editovat</span></a><span class="mw-editsection-divider"> | </span><a href="/w/index.php?title=%C4%8Cesk%C3%A9_p%C5%99edlo%C5%BEky&amp;action=edit&amp;section=7" title="Editovat zdrojový kód sekce Akuzativ (4. pád)"><span>editovat zdroj</span></a><span class="mw-editsection-bracket">]</span></span></div>
<dl><dd>pro – udělal to pro mne</dd>
<dd>za – kandidoval za demokraty, 1 porce za 5 korun</dd>
<dd>před – vynes to před dům</dd>
<dd>mimo – všichni mimo tebe, mimo terč, mimo provoz</dd>
<dd>na – dej to na stůl</dd>
<dd>pod(e) – hodit pod stůl</dd>
<dd>nad(e) – pověsit nade dveře</dd>
<dd>mezi – umístit mezi okna</dd>
<dd>skrz(e) – projít skrze zeď</dd>
<dd>přes – muž přes palubu!</dd>
<dd>o – zvýšit o 1 stupeň, zápas o 3. místo</dd>
<dd>po – být po kolena ve vodě</dd>
<dd>v – věřit v Boha</dd></dl>
<div class="mw-heading mw-heading3"><h3 id="Lokál_(6._pád)"><span id="Lok.C3.A1l_.286._p.C3.A1d.29"></span><a href="/wiki/Lok%C3%A1l" title="Lokál">Lokál</a> (6. pád)</h3><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/w/index.php?title=%C4%8Cesk%C3%A9_p%C5%99edlo%C5%BEky&amp;veaction=edit&amp;section=8" title="Editace sekce: Lokál (6. pád)" class="mw-editsection-visualeditor"><span>editovat</span></a><span class="mw-editsection-divider"> | </span><a href="/w/index.php?title=%C4%8Cesk%C3%A9_p%C5%99edlo%C5%BEky&amp;action=edit&amp;section=8" title="Editovat zdrojový kód sekce Lokál (6. pád)"><span>editovat zdroj</span></a><span class="mw-editsection-bracket">]</span></span></div>
<dl><dd>o – mluvit o tom</dd>
<dd>na – ležet na stole</dd>
<dd>v – ruka v rukávu</dd>
<dd>po – po obědě, jdu po schodech</dd>
<dd>při – při obřadu, stůj při mně</dd></dl>
<div class="mw-heading mw-heading3"><h3 id="Instrumentál_(7._pád)"><span id="Instrument.C3.A1l_.287._p.C3.A1d.29"></span><a href="/wiki/Instrument%C3%A1l" title="Instrumentál">Instrumentál</a> (7. pád)</h3><span class="mw-editsection"><span class="mw-editsection-bracket">[</span><a href="/w/index.php?title=%C4%8Cesk%C3%A9_p%C5%99edlo%C5%BEky&amp;veaction=edit&amp;section=9" title="Editace sekce: Instrumentál (7. pád)" class="mw-editsection-visualeditor"><span>editovat</span></a><span class="mw-editsection-divider"> | </span><a href="/w/index.php?title=%C4%8Cesk%C3%A9_p%C5%99edlo%C5%BEky&amp;action=edit&amp;section=9" title="Editovat zdrojový kód sekce Instrumentál (7. pád)"><span>editovat zdroj</span></a><span class="mw-editsection-bracket">]</span></span></div>
"""

# https://cs.wikipedia.org/wiki/Spojka_(slovn%C3%AD_druh)
TXT_2 = """<li><b>Spojky důsledkové</b> – <i>proto, a proto, tak, a tak, tudíž, a tudíž, tedy</i></li>
<li><b>Spojky slučovací</b> – <i>a, i, ani, nebo, či, přímo, nadto, ani–ani, jak–tak, hned–hned, jednak–jednak, zčásti–zčásti, dílem–dílem</i></li>
<li><b>Spojky odporovací</b> – <i>a, ale, avšak, však, leč, nýbrž, naopak, jenomže, jenže, sice–ale, jistě–ale</i></li>
<li><b>Spojky stupňovací</b> – <i>i, ba, ba i, ba ani, nadto, dokonce, nejen – ale i, nejen – nýbrž i</i></li>
<li><b>Spojky vylučovací</b> – <i>nebo, aneb, buď–nebo, buď–anebo</i></li>
<li><b>Spojky vysvětlovací</b> – <i>totiž, vždyť</i></li>
<li><b>Spojky příčinné</b> – <i>neboť, vždyť, totiž, však, také</i></li>
"""

TXT_3 = """aby, jakmile, až, než, nežli, zatímco, když, kdyby, pokud, protože, poněvadž, jelikož, jestliže, přestože, ačkoli, třebaže, i když, ač, že, sotva, i když"""

def remove_diacritics(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

In [34]:
prepositions = list()
for i in TXT.splitlines():
    if i.startswith("<dd>"):
        i = i.split("–")[0].replace("<dd>", "").strip()
        if "(" in i:
            prepositions.append(re.sub(r'[()]', '', i))
            prepositions.append(re.sub(r'(\(\w*\))', '', i))
        elif "/" in i:
            [prepositions.append(x) for x in i.split("/")]
        else:
            prepositions.append(i)

prepositions_diac_free = [remove_diacritics(p) for p in prepositions]
df_prepositions = pd.DataFrame({
    'prepositions': prepositions,
    'prepositions_diacritics_free': prepositions_diac_free
})
print(df_prepositions)

       prepositions prepositions_diacritics_free
0                za                           za
1              před                         pred
2               pod                          pod
3               nad                          nad
4              mezi                         mezi
5                 z                            z
6                 z                            z
7                do                           do
8               bez                          bez
9             kromě                        krome
10             krom                         krom
11            místo                        misto
12            podle                        podle
13            podél                        podel
14            kolem                        kolem
15            okolo                        okolo
16                u                            u
17            vedle                        vedle
18            během                        behem
19           pomocí 

In [35]:
conjunctions = []

for i in TXT_2.splitlines():
    i = i.split("<i>")[1]
    i = i.replace("</i></li>", "")
    i = i.split(",")
    i = [s.strip() for s in i]
    [conjunctions.append(x) for x in i]
    
conjunctions_diac_free = [remove_diacritics(c) for c in conjunctions]

conjunctions_2 = TXT_3.split(", ")
conjunctions_2_diac_free = [remove_diacritics(c) for c in conjunctions_2]

conjunctions.extend(conjunctions_2)
conjunctions_diac_free.extend(conjunctions_2_diac_free)

df_conjunctions = pd.DataFrame({
    'conjunctions': conjunctions,
    'conjunctions_diacritics_free': conjunctions_diac_free
})
print(df_conjunctions)

   conjunctions conjunctions_diacritics_free
0         proto                        proto
1       a proto                      a proto
2           tak                          tak
3         a tak                        a tak
4         tudíž                        tudiz
..          ...                          ...
66       i když                       i kdyz
67           ač                           ac
68           že                           ze
69        sotva                        sotva
70       i když                       i kdyz

[71 rows x 2 columns]


In [36]:
# Add a column to indicate the type of word
df_prepositions['type'] = 'preposition'
df_conjunctions['type'] = 'conjunction'

# Rename columns to avoid conflicts during concatenation
df_prepositions.rename(columns={'prepositions': 'word', 'prepositions_diacritics_free': 'word_diacritics_free'}, inplace=True)
df_conjunctions.rename(columns={'conjunctions': 'word', 'conjunctions_diacritics_free': 'word_diacritics_free'}, inplace=True)

# Concatenate the dataframes
df_combined = pd.concat([df_prepositions, df_conjunctions], ignore_index=True)

print(df_combined)

       word word_diacritics_free         type
0        za                   za  preposition
1      před                 pred  preposition
2       pod                  pod  preposition
3       nad                  nad  preposition
4      mezi                 mezi  preposition
..      ...                  ...          ...
115  i když               i kdyz  conjunction
116      ač                   ac  conjunction
117      že                   ze  conjunction
118   sotva                sotva  conjunction
119  i když               i kdyz  conjunction

[120 rows x 3 columns]


In [37]:
df_combined.to_csv('combined_words.csv', index=False)